In [1]:
# Import all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import warnings
import os
import subprocess
from pathlib import Path

warnings.filterwarnings('ignore')


In [5]:
# Load CSV
df = pd.read_csv(r'C:\Users\singh\Desktop\Walmart Analysis\Walmart.csv')

print(f"Dataset loaded successfully!")



Dataset loaded successfully!


In [6]:
print(f"\nDataset Shape: {df.shape} rows × {df.shape} columns")

print(f"\nColumn Names & Data Types:")
print(df.dtypes)

print(f"\nFirst 5 Rows:")
print(df.head())

print(f"\nDataset Info:")
print(df.info())

print(f"\nBasic Statistics:")
print(df.describe())


Dataset Shape: (10051, 11) rows × (10051, 11) columns

Column Names & Data Types:
invoice_id          int64
Branch             object
City               object
category           object
unit_price         object
quantity          float64
date               object
time               object
payment_method     object
rating            float64
profit_margin     float64
dtype: object

First 5 Rows:
   invoice_id   Branch         City                category unit_price  \
0           1  WALM003  San Antonio       Health and beauty     $74.69   
1           2  WALM048    Harlingen  Electronic accessories     $15.28   
2           3  WALM067  Haltom City      Home and lifestyle     $46.33   
3           4  WALM064      Bedford       Health and beauty     $58.22   
4           5  WALM013       Irving       Sports and travel     $86.31   

   quantity      date      time payment_method  rating  profit_margin  
0       7.0  05/01/19  13:08:00        Ewallet     9.1           0.48  
1       5.0  

In [7]:
print("\nDATA QUALITY ASSESSMENT")
# Duplicates
duplicate_count = df.duplicated().sum()
print(f"\n1Duplicate Rows: {duplicate_count}")

# Missing values
print(f"\nMissing Values by Column:")
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
print(missing_data[missing_data['Missing_Count'] > 0])

# Data types
print(f"\nData Types Issue:")
print(f"  'Unit price' is {df['unit_price'].dtype} (should be float)")
print(f"   Sample: {df['unit_price'].head()}")

print(f"\nSummary Before Cleaning:")
print(f"   Total Rows: {len(df):,}")
print(f"   Duplicates: {duplicate_count}")
print(f"   Total Nulls: {df.isnull().sum().sum()}")



DATA QUALITY ASSESSMENT

1Duplicate Rows: 51

Missing Values by Column:
                Column  Missing_Count  Missing_Percentage
unit_price  unit_price             31                0.31
quantity      quantity             31                0.31

Data Types Issue:
  'Unit price' is object (should be float)
   Sample: 0    $74.69
1    $15.28
2    $46.33
3    $58.22
4    $86.31
Name: unit_price, dtype: object

Summary Before Cleaning:
   Total Rows: 10,051
   Duplicates: 51
   Total Nulls: 62


In [8]:
print("\nDATA CLEANING - PHASE 1: Remove Duplicates & Nulls")

initial_rows = len(df)

# Step 1: Remove duplicates
print(f"\n1️Removing Duplicates...")
print(f"   Before: {len(df):,} rows")
df.drop_duplicates(inplace=True)
print(f"   After: {len(df):,} rows")
print(f"   Removed: {initial_rows - len(df)} duplicate rows")

# Step 2: Drop rows with missing critical columns
print(f"\nRemoving Rows with Missing Unit Price or Quantity...")
initial_rows_2 = len(df)
df.dropna(subset=['unit_price', 'quantity'], inplace=True)
print(f"   Before: {initial_rows_2:,} rows")
print(f"   After: {len(df):,} rows")
print(f"   Removed: {initial_rows_2 - len(df)} rows with missing values")

# Verify
print(f"\nVerification:")
print(f"   Duplicates remaining: {df.duplicated().sum()}")
print(f"   Total nulls remaining: {df.isnull().sum().sum()}")



DATA CLEANING - PHASE 1: Remove Duplicates & Nulls

1️Removing Duplicates...
   Before: 10,051 rows
   After: 10,000 rows
   Removed: 51 duplicate rows

Removing Rows with Missing Unit Price or Quantity...
   Before: 10,000 rows
   After: 9,969 rows
   Removed: 31 rows with missing values

Verification:
   Duplicates remaining: 0
   Total nulls remaining: 0


In [11]:
print("\nDATA CLEANING - PHASE 2: Fix Data Types")


# Fix Unit Price (remove $ and convert to float safely)
print("\n1️Fixing 'unit_price' column...")
print("   Before:", df['unit_price'].dtype)
print("   Sample:", df['unit_price'].head())

df['unit_price'] = pd.to_numeric(
    df['unit_price']
        .astype(str)
        .str.replace('$', '', regex=False),
    errors='coerce'
)

print("   After:", df['unit_price'].dtype)
print("   Sample:", df['unit_price'].head())
print("   Fixed!")

# Fix Quantity
print("\nFixing 'quantity' column...")
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
print("    Fixed!")

# Convert Date column to datetime
print("\nConverting 'date' column to datetime...")
print("   Before:", df['date'].dtype)

df['date'] = pd.to_datetime(df['date'], errors='coerce')

print("   After:", df['date'].dtype)
print("   Fixed!")

# Extract Hour from time column
print("\nExtracting 'Hour' from time column...")
df['Hour'] = pd.to_datetime(df['time'], errors='coerce').dt.hour
print("   Sample hours:", df['Hour'].head())
print("   Fixed!")

print("\nAll Data Types Fixed Successfully!")
print(df.dtypes)



DATA CLEANING - PHASE 2: Fix Data Types

1️Fixing 'unit_price' column...
   Before: float64
   Sample: 0    74.69
1    15.28
2    46.33
3    58.22
4    86.31
Name: unit_price, dtype: float64
   After: float64
   Sample: 0    74.69
1    15.28
2    46.33
3    58.22
4    86.31
Name: unit_price, dtype: float64
   Fixed!

Fixing 'quantity' column...
    Fixed!

Converting 'date' column to datetime...
   Before: datetime64[ns]
   After: datetime64[ns]
   Fixed!

Extracting 'Hour' from time column...
   Sample hours: 0    13
1    10
2    13
3    20
4    10
Name: Hour, dtype: int32
   Fixed!

All Data Types Fixed Successfully!
invoice_id                 int64
Branch                    object
City                      object
category                  object
unit_price               float64
quantity                 float64
date              datetime64[ns]
time                      object
payment_method            object
rating                   float64
profit_margin            float64
Hour     

In [12]:
print("\nFEATURE ENGINEERING: Creating New Columns")

# Create Total column
print(f"\nCreating 'Total' Column (Unit price × Quantity)...")
df['Total'] = df['unit_price'] * df['quantity']
print(f"   Sample Calculations:")
print(df[['unit_price', 'quantity', 'Total']].head(10))
print(f"Created!")

# Extract date features
print(f"\nExtracting Additional Date Features...")
df['Day_of_Week'] = df['date'].dt.day_name()
df['Month'] = df['date'].dt.month
df['Year'] = df['date'].dt.year
df['Week'] = df['date'].dt.isocalendar().week

print(f"   Added: Day_of_Week, Month, Year, Week")
print(f"   Sample:")
print(df[['date', 'Day_of_Week', 'Month', 'Year', 'Hour']].head())
print(f" Created!")

# Show final structure
print(f"\nFinal Dataset Structure:")
print(f" Shape: {df.shape}")
print(f" Columns: {list(df.columns)}")
print(f"\n{df.head()}")



FEATURE ENGINEERING: Creating New Columns

Creating 'Total' Column (Unit price × Quantity)...
   Sample Calculations:
   unit_price  quantity   Total
0       74.69       7.0  522.83
1       15.28       5.0   76.40
2       46.33       7.0  324.31
3       58.22       8.0  465.76
4       86.31       7.0  604.17
5       85.39       7.0  597.73
6       68.84       6.0  413.04
7       73.56      10.0  735.60
8       36.26       2.0   72.52
9       54.84       3.0  164.52
Created!

Extracting Additional Date Features...
   Added: Day_of_Week, Month, Year, Week
   Sample:
        date Day_of_Week  Month  Year  Hour
0 2019-05-01   Wednesday      5  2019    13
1 2019-08-03    Saturday      8  2019    10
2 2019-03-03      Sunday      3  2019    13
3 2019-01-27      Sunday      1  2019    20
4 2019-08-02      Friday      8  2019    10
 Created!

Final Dataset Structure:
 Shape: (9969, 17)
 Columns: ['invoice_id', 'Branch', 'City', 'category', 'unit_price', 'quantity', 'date', 'time', 'payment_met

In [13]:
print("\nCLEANED DATA SUMMARY")

print(f"\nKey Metrics:")
print(f"   Total Records: {len(df):,}")
print(f"   Total Columns: {len(df.columns)}")
print(f"   Date Range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"   Total Revenue: ${df['Total'].sum():,.2f}")
print(f"   Average Transaction: ${df['Total'].mean():,.2f}")
print(f"   Highest Transaction: ${df['Total'].max():,.2f}")
print(f"   Lowest Transaction: ${df['Total'].min():,.2f}")

print(f"\n Statistical Summary (Numeric Columns):")
print(df[['unit_price', 'quantity', 'Total']].describe())

# Save cleaned data
print(f"\n Saving Cleaned Data to CSV...")
df.to_csv('walmart_cleaned.csv', index=False)
print(f" Saved as 'walmart_cleaned.csv'")
print(f"   File size: {os.path.getsize('walmart_cleaned.csv') / 1024:.2f} KB")


CLEANED DATA SUMMARY

Key Metrics:
   Total Records: 9,969
   Total Columns: 17
   Date Range: 2019-01-01 to 2023-12-31
   Total Revenue: $1,209,726.38
   Average Transaction: $121.35
   Highest Transaction: $993.00
   Lowest Transaction: $10.17

 Statistical Summary (Numeric Columns):
        unit_price     quantity        Total
count  9969.000000  9969.000000  9969.000000
mean     50.622142     2.355602   121.348819
std      21.203766     1.605455   112.678040
min      10.080000     1.000000    10.170000
25%      32.000000     1.000000    54.000000
50%      51.000000     2.000000    88.000000
75%      69.000000     3.000000   156.000000
max      99.960000    10.000000   993.000000

 Saving Cleaned Data to CSV...
 Saved as 'walmart_cleaned.csv'
   File size: 1140.76 KB
